In [ ]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.7/629.7 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.3 MB/s eta 0:00:00


In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import mlflow
import mlflow.sklearn
import mlflow.pyfunc
import subprocess
import numpy as np


In [ ]:
diabetes=load_diabetes()

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(diabetes.data,diabetes.target,test_size=0.2,random_state=42)

In [ ]:
model=LinearRegression()

In [ ]:
model.fit(x_train,y_train)

LinearRegression()

In [ ]:
input_ex=np.array([[1.0]*10])

with mlflow.start_run(nested=True):
  mlflow.log_param("learning_rate",0.01)
  mlflow.log_metric("rmse",0.1)
  mlflow.sklearn.log_model(model,"model", input_example=input_ex)

In [ ]:
#print(mlflow.get_artifact_uri('model'
from mlflow.tracking import MlflowClient

client=MlflowClient()
ex_id="0"

runs=client.search_runs(experiment_ids=[ex_id],order_by=['attributes.start_time desc'])
print(f"MOdel uri's for ex_id {ex_id}:")
for run in runs:
  run_id=run.info.run_id
  model_uri=f'runs:/{run_id}/model'
  print(f"run_id:{run_id}, Model URI: {model_uri}")

MOdel uri's for ex_id 0:
run_id:dc3e2b49003b4c1c8f176e062555bde2, Model URI: runs:/dc3e2b49003b4c1c8f176e062555bde2/model


In [ ]:
model_uri="runs:/dc3e2b49003b4c1c8f176e062555bde2/model"
model=mlflow.pyfunc.load_model(model_uri)

In [ ]:
input_array=np.array([[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0]])
pred=model.predict(input_array)

In [ ]:
print(pred)

[11633.33138298]


In [ ]:
model_uri="runs:/aff6ccef50404c8ba4fb7d362eee7coa/model"
host = "0.0.0.0"
port = 8000
command = "mlflow","models","serve","--model-uri",model_uri,
"--host",host,"--port",str(port)

subprocess.run(command)



CompletedProcess(args=('mlflow', 'models', 'serve', '--model-uri', 'runs:/aff6ccef50404c8ba4fb7d362eee7coa/model'), returncode=1)

In [ ]:
import requests
import time
data= {"data":[[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0]]}
url= "http://localhost:8000/invocations"
retries=5
for attempt in range(1,retries+1):
  try:
    response = requests.post(url,json=data)
    if response.status_code==200:
      print("predictions:",response.json())
      break
    else:
      print(f"error: {response.status_code},{response.text}")
      if attempt<retries:
        print(f"Retries....({attempt}/{retries})")
        time.sleep(5)
      else:
        print("max retries exceeded.could not the model server.")
  except requests.exceptions.RequestException as e:
    print(f"request error: {e}")

request error: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /invocations (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c3f3e033610>: Failed to establish a new connection: [Errno 111] Connection refused'))
request error: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /invocations (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c3f3e032dd0>: Failed to establish a new connection: [Errno 111] Connection refused'))
request error: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /invocations (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c3f3dfefaf0>: Failed to establish a new connection: [Errno 111] Connection refused'))
request error: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /invocations (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object 

**07/01/2025**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler

In [ ]:
diabetes_data = load_diabetes()
feature_names = diabetes_data.feature_names
df = pd.DataFrame(data=diabetes_data.data, columns=feature_names)
df['target'] = diabetes_data.target

In [ ]:
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


In [ ]:
df.shape

(442, 11)

In [ ]:
df.size

4862

In [ ]:
df.isnull().sum()

,0
age,0
sex,0
bmi,0
bp,0
s1,0
s2,0
s3,0
s4,0
s5,0
s6,0


In [ ]:
df['target']

,target
0,151.0
1,75.0
2,141.0
3,206.0
4,135.0
...,...
437,178.0
438,104.0
439,132.0
440,220.0


In [ ]:
model=LinearRegression()

In [ ]:
x=df.drop(['target'],axis=1)
y=df['target']

In [ ]:
scaler=StandardScaler()
y=scaler.fit_transform(y.reshape(-1,1))

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

In [ ]:
model.fit(x_train,y_train)

LinearRegression()

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
mse=mean_squared_error(y_test,y_pred)
mse

0.4803315241972985